<a href="https://colab.research.google.com/github/RGologorsky/fastmri/blob/master/common_archs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# conv layer consists of 2dconv + batchnorm + activation ReLU
def conv(ni, nf, ks=3, act=True):
  layers = [nn.Conv2d(ni, nf, stride=2, kernel_size=ks, padding=ks//2)]
  layers.append(nn.BatchNorm2d(nf))
  if act: layers.append(nn.ReLU())
  return nn.Sequential(*layers)

# halve x w/ceiling (match fastai stride 2 layers)
def halve(x): return (x+1)//2

# simple cnn = stride 2, 3x3 kernel conv layers
def get_simple_cnn(c_in, n_out, sz, intro_nf=8, intro_ks=5):
  
  # init layers
  layers = []
  curr_ni, curr_sz = c_in, sz

  # add intro layer (stride 2)
  if halve(curr_sz) > 1: 
    layers.append(conv(curr_ni, nf=intro_nf, ks=intro_ks))
    curr_ni, curr_sz = intro_nf, halve(curr_sz)
  
  # add middle layers (stride 2: double #input features, halve size)
  while halve(curr_sz) > 1:
    layers.append(conv(curr_ni, curr_ni*2))
    curr_ni, curr_sz = curr_ni*2, halve(curr_sz)
  
  # add final stride-2 output layer & flatten
  layers += [conv(curr_ni, n_out, act=False), Flatten()]
  return nn.Sequential(*layers)